In [161]:
import pandas as pd
import scipy.stats as stats
from scipy.stats import chi2_contingency

In [162]:
users = pd.read_csv('ab_users_data.csv', parse_dates = ['time'])
products = pd.read_csv('ab_products.csv')
orders = pd.read_csv('ab_orders.csv', parse_dates = ['creation_time'])

In [163]:
users.head(7)

,user_id,order_id,action,time,date,group
0,964,1255,create_order,2022-08-26 00:00:19,2022-08-26,0
1,965,1256,create_order,2022-08-26 00:02:21,2022-08-26,1
2,964,1257,create_order,2022-08-26 00:02:27,2022-08-26,0
3,966,1258,create_order,2022-08-26 00:02:56,2022-08-26,0
4,967,1259,create_order,2022-08-26 00:03:37,2022-08-26,1
5,968,1261,create_order,2022-08-26 00:05:35,2022-08-26,0
6,969,1262,create_order,2022-08-26 00:06:25,2022-08-26,0


In [164]:
products.head(7)

,product_id,name,price
0,1,сахар,150.0
1,2,чай зеленый в пакетиках,50.0
2,3,вода негазированная,80.4
3,4,леденцы,45.5
4,5,кофе 3 в 1,15.0
5,6,сухарики,25.0
6,7,кофе зерновой,350.8


In [165]:
orders.head(7)

,order_id,creation_time,product_ids
0,1255,2022-08-26 00:00:19,"{75, 22, 53, 84}"
1,1256,2022-08-26 00:02:21,"{56, 76, 39}"
2,1257,2022-08-26 00:02:27,"{76, 34, 41, 38}"
3,1258,2022-08-26 00:02:56,"{74, 6}"
4,1259,2022-08-26 00:03:37,"{20, 45, 67, 26}"
5,1261,2022-08-26 00:05:35,"{30, 35, 69, 6}"
6,1262,2022-08-26 00:06:25,"{23, 69, 45, 33}"


### Проанализируем каждый из датасетов
#### ab_users_data.csv

In [166]:
users.shape

(4337, 6)

In [167]:
users.dtypes

user_id              int64
order_id             int64
action              object
time        datetime64[ns]
date                object
group                int64
dtype: object

In [168]:
users.isna().sum()

user_id     0
order_id    0
action      0
time        0
date        0
group       0
dtype: int64

In [169]:
users.action.unique()

array(['create_order', 'cancel_order'], dtype=object)

In [170]:
users.query('action == "cancel_order"')

,user_id,order_id,action,time,date,group
8,966,1258,cancel_order,2022-08-26 00:08:25.486419,2022-08-26,0
39,993,1296,cancel_order,2022-08-26 00:44:18.150475,2022-08-26,0
47,995,1298,cancel_order,2022-08-26 00:49:51.514240,2022-08-26,0
54,1002,1308,cancel_order,2022-08-26 00:58:01.924112,2022-08-26,1
60,1008,1315,cancel_order,2022-08-26 01:06:11.292953,2022-08-26,0
...,...,...,...,...,...,...
4221,1498,54938,cancel_order,2022-09-08 06:46:29.885081,2022-09-08,0
4255,1658,56612,cancel_order,2022-09-08 13:31:37.085777,2022-09-08,0
4259,1314,56772,cancel_order,2022-09-08 14:09:53.652063,2022-09-08,1
4307,988,58492,cancel_order,2022-09-08 20:07:22.918779,2022-09-08,1


#### ab_products.csv

In [171]:
products.shape

(87, 3)

In [172]:
products.dtypes

product_id      int64
name           object
price         float64
dtype: object

In [173]:
products.isna().sum()

product_id    0
name          0
price         0
dtype: int64

#### ab_orders.csv

In [174]:
orders.shape

(4123, 3)

In [175]:
orders.dtypes

order_id                  int64
creation_time    datetime64[ns]
product_ids              object
dtype: object

In [176]:
orders.isna().sum()

order_id         0
creation_time    0
product_ids      0
dtype: int64

### Как оценить пользу системы рекомендаций для бизнеса и пользователей?
Определим для этого метрики:
* Первая метрика - количество товаров в заказе. Если система работает правильно, то пользователь выберет больше товаров.
* Вторая метрика - доход с заказа (средний чек). Если система работает правильно, то бизнес получит больще дохода с каждого заказа (средний чек увеличится)
* Третья метрика - количество отмененных заказов. Система работает лучше --> отказов меньше. 
    *(а может быть и нет)*
* Четвертая метрика - количество заказов на пользователя. Система работает --> пользователь выберет больше товаров.

### Начнем с третьей метрики

Почему именно с нее? Т.к. в ней меньше всего уверенности, то начнем с нее, чтобы просто закрыть ее.

In [177]:
# считаем количество отменненых и созданных заказов
canceled_perc = users.groupby(['group', 'action'], as_index = False).agg({'order_id': 'count'})
canceled_perc

,group,action,order_id
0,0,cancel_order,82
1,0,create_order,1609
2,1,cancel_order,132
3,1,create_order,2514


In [178]:
canceled_g0 = (canceled_perc.order_id[0] / (canceled_perc.order_id[1])) * 100
canceled_g1 = (canceled_perc.order_id[2] / (canceled_perc.order_id[3])) * 100
print(f'Percentage of canceled orders in 0-group: {round(canceled_g0, 2)}')
print(f'Percentage of canceled orders in 1-group: {round(canceled_g1, 2)}')

Percentage of canceled orders in 0-group: 5.1
Percentage of canceled orders in 1-group: 5.25


Доля отмененных заказов немного выросла.

Попробуем определить, является ли это изменение статзначимым.

Для этого составим таблицу сопряженности и применим Хи2



In [179]:
df1 = pd.DataFrame({'canceled':[canceled_perc.order_id[0], canceled_perc.order_id[2]],
                    'created':[canceled_perc.order_id[1], canceled_perc.order_id[3]]})
df1.index.name = 'group'
df1

,canceled,created
group,,
0,82,1609
1,132,2514


### Какие проверяем гипотезы?
* Н0 - Количество (доли) отмененных заказов в группах равны
* Н1 - Имеется значимое различие в количестве отмененных заказов в группах 

*Чтобы отвергнуть Н0 необходимо иметь уровень значимости меньше 0.05*

In [180]:
result = chi2_contingency(df1)
print(f'statistic: {result[0]}')
print(f'p-value: {result[1]}')

statistic: 0.018211165651942023
p-value: 0.8926523935841298


### Вывод по третьей метрике: 
Уровень значимости оказался выше порогового значения --> статзначимого различия в количестве отменненых заказов нет

*Другими словами новая система никак не повлияла на количество отказов*

### Перейдем к первой метрике - количество купленных заказов.

Для этого оставим в таблице только те заказы, которые не были отменены.

In [181]:
canceled_orders = users.query('action == "cancel_order"').order_id

created_orders = users.query('order_id not in @canceled_orders')
created_orders

,user_id,order_id,action,time,date,group
0,964,1255,create_order,2022-08-26 00:00:19,2022-08-26,0
1,965,1256,create_order,2022-08-26 00:02:21,2022-08-26,1
2,964,1257,create_order,2022-08-26 00:02:27,2022-08-26,0
4,967,1259,create_order,2022-08-26 00:03:37,2022-08-26,1
5,968,1261,create_order,2022-08-26 00:05:35,2022-08-26,0
...,...,...,...,...,...,...
4332,990,59422,create_order,2022-09-08 23:13:03,2022-09-08,1
4333,1418,59439,create_order,2022-09-08 23:18:05,2022-09-08,1
4334,1605,59464,create_order,2022-09-08 23:22:27,2022-09-08,0
4335,1461,59487,create_order,2022-09-08 23:29:06,2022-09-08,0


In [182]:
# теперь необходимо объединить таблицу users c orders
data = created_orders.merge(orders, on = 'order_id')
data.head(7)

,user_id,order_id,action,time,date,group,creation_time,product_ids
0,964,1255,create_order,2022-08-26 00:00:19,2022-08-26,0,2022-08-26 00:00:19,"{75, 22, 53, 84}"
1,965,1256,create_order,2022-08-26 00:02:21,2022-08-26,1,2022-08-26 00:02:21,"{56, 76, 39}"
2,964,1257,create_order,2022-08-26 00:02:27,2022-08-26,0,2022-08-26 00:02:27,"{76, 34, 41, 38}"
3,967,1259,create_order,2022-08-26 00:03:37,2022-08-26,1,2022-08-26 00:03:37,"{20, 45, 67, 26}"
4,968,1261,create_order,2022-08-26 00:05:35,2022-08-26,0,2022-08-26 00:05:35,"{30, 35, 69, 6}"
5,969,1262,create_order,2022-08-26 00:06:25,2022-08-26,0,2022-08-26 00:06:25,"{23, 69, 45, 33}"
6,970,1263,create_order,2022-08-26 00:08:22,2022-08-26,1,2022-08-26 00:08:22,"{22, 23, 5}"


In [183]:
# для более удобной работы необходимо преобразовать колонку product_ids
data.product_ids = data.product_ids.apply(lambda x: x.replace('{', '').replace('}', '').split(', '))
data

,user_id,order_id,action,time,date,group,creation_time,product_ids
0,964,1255,create_order,2022-08-26 00:00:19,2022-08-26,0,2022-08-26 00:00:19,"[75, 22, 53, 84]"
1,965,1256,create_order,2022-08-26 00:02:21,2022-08-26,1,2022-08-26 00:02:21,"[56, 76, 39]"
2,964,1257,create_order,2022-08-26 00:02:27,2022-08-26,0,2022-08-26 00:02:27,"[76, 34, 41, 38]"
3,967,1259,create_order,2022-08-26 00:03:37,2022-08-26,1,2022-08-26 00:03:37,"[20, 45, 67, 26]"
4,968,1261,create_order,2022-08-26 00:05:35,2022-08-26,0,2022-08-26 00:05:35,"[30, 35, 69, 6]"
...,...,...,...,...,...,...,...,...
3904,990,59422,create_order,2022-09-08 23:13:03,2022-09-08,1,2022-09-08 23:13:03,"[84, 8, 24]"
3905,1418,59439,create_order,2022-09-08 23:18:05,2022-09-08,1,2022-09-08 23:18:05,"[9, 25, 75, 30, 6]"
3906,1605,59464,create_order,2022-09-08 23:22:27,2022-09-08,0,2022-09-08 23:22:27,"[60, 41, 46]"
3907,1461,59487,create_order,2022-09-08 23:29:06,2022-09-08,0,2022-09-08 23:29:06,"[9, 62, 77]"


In [184]:
data['goods_number'] = data.product_ids.apply(lambda x: len(x))
data

,user_id,order_id,action,time,date,group,creation_time,product_ids,goods_number
0,964,1255,create_order,2022-08-26 00:00:19,2022-08-26,0,2022-08-26 00:00:19,"[75, 22, 53, 84]",4
1,965,1256,create_order,2022-08-26 00:02:21,2022-08-26,1,2022-08-26 00:02:21,"[56, 76, 39]",3
2,964,1257,create_order,2022-08-26 00:02:27,2022-08-26,0,2022-08-26 00:02:27,"[76, 34, 41, 38]",4
3,967,1259,create_order,2022-08-26 00:03:37,2022-08-26,1,2022-08-26 00:03:37,"[20, 45, 67, 26]",4
4,968,1261,create_order,2022-08-26 00:05:35,2022-08-26,0,2022-08-26 00:05:35,"[30, 35, 69, 6]",4
...,...,...,...,...,...,...,...,...,...
3904,990,59422,create_order,2022-09-08 23:13:03,2022-09-08,1,2022-09-08 23:13:03,"[84, 8, 24]",3
3905,1418,59439,create_order,2022-09-08 23:18:05,2022-09-08,1,2022-09-08 23:18:05,"[9, 25, 75, 30, 6]",5
3906,1605,59464,create_order,2022-09-08 23:22:27,2022-09-08,0,2022-09-08 23:22:27,"[60, 41, 46]",3
3907,1461,59487,create_order,2022-09-08 23:29:06,2022-09-08,0,2022-09-08 23:29:06,"[9, 62, 77]",3


In [185]:
# посчитаем, как изменилось среднее количесвто товаров в заказах в разбивке по группам
data.groupby('group').agg({'goods_number':'mean'})

,goods_number
group,
0,3.341847
1,3.350126


Среднее количество покупаемых товаров совсем немного увеличилось.

*Имеет ли смысл проверять такое изменение на значимость?*

Тем не менее... Проверим)

**И снова вернемся к гипотезам...**
* Н0 - среднее количество покупаемых товаров не изменилось
* Н1 - среднее количество покупаемых товаров значимо изменилось

*Чтобы отвергнуть Н0 необходимо иметь уровень значимости меньше 0.05*

In [186]:
group_0 = data.query('group == 0').goods_number
group_1 = data.query('group == 1').goods_number

In [187]:
# Проведем t-тест
stats.ttest_ind(group_1, group_0, equal_var=True)

Ttest_indResult(statistic=0.19638057741712578, pvalue=0.84432251088501)

Как мы можем заметить, снова уровень значимости оказался сильно выше порогового значения. Следовательно среднее количество покупаемых товаров в заказе не изменилось.

*То есть, новая система не влияет на количество покупаемых товаров*

### Перейдем ко второй метрике

Проверим, как меняется средний чек до и после введения системы рекомендаций

*Но для этого снова придется немного подготовить данные*

In [188]:
full_data = data.drop(columns = ['creation_time', 'goods_number'])
full_data = full_data.explode('product_ids')
full_data.product_ids = full_data.product_ids.apply(lambda x: int(x))
full_data = full_data.rename(columns = {'product_ids':'product_id'})
full_data

,user_id,order_id,action,time,date,group,product_id
0,964,1255,create_order,2022-08-26 00:00:19,2022-08-26,0,75
0,964,1255,create_order,2022-08-26 00:00:19,2022-08-26,0,22
0,964,1255,create_order,2022-08-26 00:00:19,2022-08-26,0,53
0,964,1255,create_order,2022-08-26 00:00:19,2022-08-26,0,84
1,965,1256,create_order,2022-08-26 00:02:21,2022-08-26,1,56
...,...,...,...,...,...,...,...
3907,1461,59487,create_order,2022-09-08 23:29:06,2022-09-08,0,9
3907,1461,59487,create_order,2022-09-08 23:29:06,2022-09-08,0,62
3907,1461,59487,create_order,2022-09-08 23:29:06,2022-09-08,0,77
3908,1376,59533,create_order,2022-09-08 23:41:24,2022-09-08,0,17


In [189]:
full_data = full_data.merge(products, on = 'product_id')
full_data

,user_id,order_id,action,time,date,group,product_id,name,price
0,964,1255,create_order,2022-08-26 00:00:19,2022-08-26,0,75,сок ананасовый,120.0
1,964,1255,create_order,2022-08-26 00:00:19,2022-08-26,0,22,сок мультифрукт,120.0
2,964,1255,create_order,2022-08-26 00:00:19,2022-08-26,0,53,мука,78.3
3,964,1255,create_order,2022-08-26 00:00:19,2022-08-26,0,84,мандарины,90.4
4,965,1256,create_order,2022-08-26 00:02:21,2022-08-26,1,56,сосиски,150.0
...,...,...,...,...,...,...,...,...,...
13078,1461,59487,create_order,2022-09-08 23:29:06,2022-09-08,0,9,чай черный листовой,83.5
13079,1461,59487,create_order,2022-09-08 23:29:06,2022-09-08,0,62,сок яблочный,120.0
13080,1461,59487,create_order,2022-09-08 23:29:06,2022-09-08,0,77,курица,298.4
13081,1376,59533,create_order,2022-09-08 23:41:24,2022-09-08,0,17,морс брусничный,190.0


In [197]:
full_data.groupby('group').agg({'price':'sum'})

,price
group,
0,583455.9
1,878628.4


Можем заметить, что выручка в группе с системой рекомендации выросла.

Проверим средний чек на пользователя.

In [200]:
mean_check = full_data.groupby(['group', 'user_id', 'order_id'], as_index = False).agg({'price':'sum'})
mean_check.groupby('group').agg({'price':'mean'})

,price
group,
0,382.092927
1,368.861629


Можем заметить, что средний чек немного уменьшился.

Проверим такое изменение на значимость.

#### Гипотезы:
* Н0 - различий в средних чеках нет
* Н1 - различия в средних чеках есть

*Чтобы отвергнуть Н0 необходимо иметь уровень значимости меньше 0.05*

In [202]:
g0 = mean_check.query('group == 0').price
g1 = mean_check.query('group == 1').price

In [203]:
stats.ttest_ind(g0, g1)

Ttest_indResult(statistic=1.6334875192269538, pvalue=0.10244707728234388)

И снова уровень значимости оказался выше порогового значения. Следовательно система не влияет на средний чек

### Проверим четвертую метрику

(Количество товаров на человека)

In [205]:
goods_user = full_data.groupby(['group', 'user_id'], as_index = False).agg({'order_id':'nunique'})
goods_user.groupby(['group']).agg({'order_id':'mean'})

,order_id
group,
0,2.982422
1,4.754491


Можем заметить, что в данном случае количество товаров на человека заметно возросло

Проверим это изменение на значимость

#### Гипотезы

* Н0 - среднее количество товаров в группах не отличается
* Н1 - среднее количество товаров в группах отличается

*Чтобы отвергнуть Н0 необходимо иметь уровень значимости меньше 0.05*

In [208]:
gr0 = goods_user.query('group == 0').order_id
gr1 = goods_user.query('group == 1').order_id

In [209]:
stats.ttest_ind(gr0, gr1)

Ttest_indResult(statistic=-13.993124788985595, pvalue=8.430777321728061e-41)

Уровень значимости сильно ниже порогового значения, что позволяет нам отбросить гипотезу Н0.

*То есть система рекомендаций позволила повысить среднее количество товаров в заказе на человека*

## Выводы:
* Разработанная система не повлияла на следующие показатели: средний чек, количество отмененных заказов, количество товаров в заказе
* Но разработанная система увеличила количество товаров на пользователя


